In [88]:
from src.eda.soporte_eda import *
import pandas as pd

In [73]:
conectar_bd()

(<connection object at 0x0000015CC2CF4E10; dsn: 'user=postgres password=xxx dbname=Proyecto_Sephora host=localhost port=5432', closed: 0>,
 <cursor object at 0x0000015CC2CEA880; closed: 0>)

## Análisis General Sephora

Se parte de una query que trae los aspectos más relevantes de los productos, es decir, lo que les identifica (la url), el nombre, su descripción, la marca, la categoría y usbcategoría a la que pertenecen, el éxito que tienen (a partir del numero de valoraciones y la valoración) y el numero de variaciones de producto. Otras características consideradas más secundarias y menos relevantes sobre los productos son el acabado, la textura, la responsabilidad etc. Características que se emplearán en el análisis más específico para poder concretar mejor en el producto final recomendado a lanzar por Sephora Collection, pero que en este primer análisis general no se tomarán en cuenta, pues este análisis simplemente tiene en cuenta poder obtener una primera aproximación sobre la tienda de Sephora y sobretodo, poder comenzar a posicionar a Sephora Collection en comparación con el resto de las marcas. 

In [296]:
query_general = """
select 
    p.url_producto,
    p.nombre,
    p.descripcion,
    m.nombre_marca, 
    c.nombre_categoria,
    s.nombre_subcategoria,
    h.precio,
    h.numero_valoraciones,
    h.valoracion,
    h.numero_variaciones
from productos p 
join marcas m on p.id_marca = m.id_marca
left join categorias c on p.id_categoria = c.id_categoria
left join subcategorias s on p.id_subcategoria = s.id_subcategoria
join historico h on p.id_producto = h.id_producto;
"""

In [297]:
df = ejecutar_query(query_general)
df.head()

,url_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
0,https://www.sephora.es/p/soft-lit-naturally-lu...,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7,50
1,https://www.sephora.es/p/triclone-skin-tech-fo...,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5,51
2,https://www.sephora.es/p/triclone-skin-tech-hy...,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1,30
3,https://www.sephora.es/p/color-fuse-talc-free-...,Color Fuse Talc-Free Powder Blush With Ferment...,Colorete en polvo,Haus Labs,Rostro,Colorete,32.99,18,4.8,6
4,https://www.sephora.es/p/bio-radiant-gel-powde...,Bio-Radiant Gel-Powder Highlighter,Iluminador con arnica fermentada,Haus Labs,Rostro,Iluminador,46.99,8,4.8,5


In [78]:
df.shape

(1829, 10)

De primeras se cuenta con las 10 columnas ya previamente mencionadas y 1829 filas, es decir, 1829 productos de maquillaje en todo Sephora.

In [79]:
def info_reporte(dataframe):
    """
    Genera un informe sobre los valores nulos y tipos de datos de un DataFrame.

    Args:
        dataframe (pd.DataFrame): El DataFrame a analizar.

    Returns:
        pd.DataFrame: Un DataFrame con el número y porcentaje de valores nulos por columna,
                      junto con el tipo de dato de cada columna.
    """
    df_report = pd.DataFrame()
    df_report["Numero_nulos"] = dataframe.isnull().sum()
    df_report["Porcentaje_nulos"] = round((dataframe.isnull().sum()/dataframe.shape[0]*100), 2)
    df_report["Tipo_dato"] = dataframe.dtypes
    return df_report

In [80]:
info_reporte(df)

,Numero_nulos,Porcentaje_nulos,Tipo_dato
url_producto,0,0.00,object
nombre,3,0.16,object
descripcion,98,5.36,object
nombre_marca,0,0.00,object
nombre_categoria,9,0.49,object
nombre_subcategoria,33,1.80,object
precio,0,0.00,float64
numero_valoraciones,0,0.00,int64
valoracion,0,0.00,float64
numero_variaciones,0,0.00,int64


Con respecto a los valores nulos, encontramos 3 productos con nulos en el nombre, 98 con nulos en la descripción, 9 con nulos en la categoría y 33 con nulos en la subcategoría. Estos nulos no son necesarios tratarlos ya que son nulos que tienen que estar ahí, pues correctamente hay determinados productos de los cuales se desconoce su nombre, su descripción, su categoría o su subcategoría. De todas formas, el % de nulos que hay en cada una de estas columnas es mínimo por lo que se van a mantener estos productos ya que el resto de información puede resultar relevante para el análisis. Por otro lado, en cuanto al tipo de dato, en la extracción hay un proceso de transformación asegurando que cada columna se extraiga con el tipo de dato correcto y por lo tanto, todas las columnas tienen el tipo de dato correcto.

In [81]:
df["url_producto"].duplicated().sum()

np.int64(7)

In [84]:
df.drop_duplicates(subset=["url_producto"], inplace=True)
df.shape

(1822, 10)

En cuanto a los duplicados, no debería haber ningún duplicado por el campo de url del producto, ya que este campo es el identificador único de los productos. No debería haber ningún duplicado porque al haber realizado únicamente un scrapeo (el primero), los productos no deben aparecer más de una vez en la tabla de histórico. Por ahora no se ha logrado identificar en el código donde puede estar el punto donde se comete este error generando duplicados sobre 7 productos (identifica como que los productos ya están en la tabla "productos" y por lo tanto inserta una nueva fila a la tabla "historico", convirtiéndose en un duplicado erróneo). De momento, y al ser muy pocos los duplicados, para poder realizar un análisis con los datos correctamente recogidos, elimino directamente las filas duplicadas y por lo tanto nos quedamos con el número correcto de productos de maquillaje en toda Sephora, que son 1822. 

### Número total de productos en Sephora

In [85]:
numero_total_productos = df.shape[0]
print(f"Numero total de productos: {numero_total_productos}")

Numero total de productos: 1822


In [86]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,1822.0,37.201850,18.895532,3.99,25.0,34.99,45.00,169.0
numero_valoraciones,1822.0,590.726674,1707.459476,0.00,3.0,42.50,406.75,20035.0
valoracion,1822.0,3.799671,1.543679,0.00,3.9,4.40,4.70,5.0
numero_variaciones,1822.0,4.249177,7.088864,0.00,0.0,2.00,6.00,60.0


Descriptivos de las variables numéricas: 
- Precio: El precio de los productos en Sephora oscila entre 3,99 € y 169 €. La media se sitúa en 37,20 €, mientras que la mediana es de 34,99 €, lo que indica que la distribución está sesgada a la derecha. Esto significa que hay una mayoría de productos con precios moderados, pero algunos artículos significativamente más caros elevan el promedio. El 25% de los productos cuesta menos de 25 €, el 50% menos de 34,99 €, y el 75% menos de 45 €. Solo el 25% de los productos supera los 45 €, y un pequeño número alcanza precios muy altos, lo que sugiere la existencia de productos premium que pueden considerarse outliers dentro de la gama de precios.
- Número de valoraciones: La cantidad de valoraciones por producto varía de forma muy amplia, desde 0 hasta 20.035. La media es de 590,73 valoraciones, pero la mediana es de solo 42,5, lo que refleja una distribución altamente sesgada a la derecha. Esto indica que la mayoría de los productos tiene muy pocas valoraciones, mientras que unos pocos concentran una gran cantidad, elevando considerablemente el promedio. Aproximadamente el 25% de los productos cuenta con menos de 3 valoraciones, la mitad tiene menos de 42,5, y el 75% no supera las 406,75 valoraciones. Solo una cuarta parte de los productos supera ese umbral, y algunos alcanzan cifras muy altas, evidenciando casos claramente atípicos. Esta distribución sugiere que la visibilidad o la popularidad está muy concentrada en unos pocos productos dentro del catálogo.
- Valoración: Las valoraciones de los productos se mueven entre 0 y 5 puntos. La media es de 3,80, mientras que la mediana se sitúa en 4,40, lo que revela una ligera asimetría hacia la izquierda. Esto sugiere que la mayoría de los productos recibe buenas puntuaciones, pero algunos obtienen valoraciones muy bajas, lo que reduce el promedio. Un cuarto de los productos tiene una valoración inferior a 3,9, mientras que el 50% no supera los 4,4 y el 75% se sitúa por debajo de 4,7. Solo una pequeña proporción logra valoraciones superiores a ese nivel, alcanzando incluso el máximo de 5. Por lo tanto, aunque las valoraciones son en general altas, existen productos con puntuaciones bajas que podrían indicar problemas de calidad o satisfacción del cliente, y que pueden considerarse outliers negativos.
- El número de variaciones disponibles por producto, como diferentes tonos o tamaños, varía desde 0 hasta 60. La media es de 4,25 variaciones por producto, mientras que la mediana es de 2, lo cual indica una distribución fuertemente sesgada a la derecha. Esto significa que la mayoría de los productos tiene pocas opciones, pero algunos ofrecen una amplia gama de variaciones, lo que eleva la media general. El 25% de los productos no tiene ninguna variación, la mitad tiene dos o menos, y el 75% no supera las seis variaciones. Solo una minoría supera este número, y unos pocos productos llegan a tener hasta 60, situándose claramente como outliers. Esta situación es habitual en productos como bases de maquillaje, pintalabios o correctores, donde se requiere una gran diversidad de tonos para adaptarse a diferentes perfiles de consumidores.

### Valores representativos de las variables numéricas

En este análisis, se ha optado por utilizar la mediana en lugar de la media como medida representativa de las variables numéricas (precio, número de valoraciones, valoración y número de variaciones). Esta decisión se debe a que los datos presentan distribuciones claramente sesgadas y la presencia de valores atípicos (outliers) que distorsionan significativamente la media. Por tanto, el uso de la mediana permite describir con mayor precisión el comportamiento típico de los productos en Sephora, evitando que unos pocos valores extremos condicionen el análisis. Es una elección más robusta y fiel a la realidad de la distribución general del catálogo.

In [109]:
mediana_precio = df["precio"].median()
print(f"Mediana precio: {mediana_precio}")
mediana_num_valoraciones = df["numero_valoraciones"].median()
print(f"Mediana numero valoraciones: {mediana_num_valoraciones}")
mediana_valoracion = df["valoracion"].median()
print(f"Mediana valoracion: {mediana_valoracion}")
mediana_num_variaciones = df["numero_variaciones"].median()
print(f"Mediana numero variaciones: {mediana_num_variaciones}")

Mediana precio: 34.99
Mediana numero valoraciones: 42.5
Mediana valoracion: 4.4
Mediana numero variaciones: 2.0


### Observación de outliers de variables numéricas

In [117]:
def detectar_outliers(df, columna, modo='superior'):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    umbral_inferior = Q1 - 1.5 * IQR
    umbral_superior = Q3 + 1.5 * IQR

    if modo == 'superior':
        outliers = df[df[columna] > umbral_superior]
    else:
        outliers = df[df[columna] < umbral_inferior]

    return outliers[['nombre', 'descripcion', 'nombre_marca', 'nombre_categoria', 'nombre_subcategoria', 'precio', 'numero_valoraciones', 'valoracion', 'numero_variaciones']]

outliers_precio = detectar_outliers(df, 'precio', modo='superior')
outliers_valoraciones = detectar_outliers(df, 'numero_valoraciones', modo='superior')
outliers_valoracion = detectar_outliers(df, 'valoracion', modo='inferior')
outliers_variaciones = detectar_outliers(df, 'numero_variaciones', modo='superior')

In [118]:
outliers_precio.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
12,HD Skin All-In-One Palette,Paleta de maquillaje todo en uno,Make Up For Ever,Paletas maquillaje,Paleta rostro,85.0,263,4.0,2
15,Diorshow 5 Couleurs edición limitada cannage,Paleta de sombras de ojos,Dior,Ojos,Paleta de sombras de ojos,76.0,0,0.0,4
22,Invisible UV Flawless Poreless Primer,Base de maquillaje,Charlotte Tilbury,Rostro,Primer & spray fijador,79.0,3,4.7,0
101,Green Flash,Kit pro Hortencia & Poppy Red,Manucurist,Estuches maquillaje,Estuches de manicura,86.0,0,0.0,0
189,HD Skin Sculting Palette,Paleta Modeladora del rostro Crema,Make Up For Ever,Paletas maquillaje,Paleta rostro,84.0,179,4.6,0


In [119]:
outliers_precio.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,56.0,103.462321,23.587274,76.0,85.0,96.00,115.2475,169.0
numero_valoraciones,56.0,101.267857,282.616701,0.0,2.0,16.50,44.2500,1959.0
valoracion,56.0,3.800000,1.824978,0.0,4.0,4.65,4.9000,5.0
numero_variaciones,56.0,3.196429,5.955653,0.0,0.0,0.00,4.0000,30.0


In [120]:
outliers_precio.describe(include='object').T

,count,unique,top,freq
nombre,56,54,Green Flash,2
descripcion,51,39,Paleta de sombras de ojos,7
nombre_marca,56,15,Sisley,19
nombre_categoria,55,7,Rostro,27
nombre_subcategoria,55,16,Base de maquillaje,12


Los productos outliers por precio en Sephora tienen un precio significativamente más alto, con una mediana de 96.00, en comparación con los productos generales, cuya mediana es de 34.99. Además, estos productos también tienen mejores valoraciones, con una mediana de 4.65, superior a la mediana general de 4.40. En cuanto al número de valoraciones, los outliers tienen una mediana de 16.50, menor a la mediana de 42.50 de los productos generales. En términos de variaciones, la mediana de los outliers es de 0.00, indicando que estos productos tienen menos opciones disponibles en comparación con los productos generales, cuya mediana de variaciones es de 2.00. Estos productos provienen principalmente de marcas de lujo como Sisley y se concentran en las categorías de Rostro y Base de maquillaje. En resumen, los productos más caros de Sephora tienden a tener mejores valoraciones, menos valoraciones y menos opciones disponibles en comparación con los productos más económicos.

In [121]:
outliers_valoraciones.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
34,Vanish™ Airbrush Concealer,Corrector antiojeras,Hourglass,Rostro,Antiojeras & corrector,42.00,3247,4.3,29
43,Pout Preserve Lip Treatment,Cuidado labial hidratante y antiedad,Olehenriksen,Labios,Bálsamo labial,21.00,1185,4.4,8
54,Limtless Lash Mascara,Mascara,Ilia,Ojos,Máscara de pestañas,35.00,4117,4.5,2
76,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...",Valentino,Labios,Gloss/ Brillo labial,39.99,1020,4.7,6
89,Easy Bake Setting Spray,Spray fijador de maquillaje 16 horas,Huda Beauty,Rostro,Primer & spray fijador,35.00,1014,4.6,2


In [122]:
outliers_valoraciones.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,256.0,36.228828,12.815420,9.99,27.7425,35.0,42.0,79.0
numero_valoraciones,256.0,3380.621094,3378.514686,1014.00,1397.5000,2292.5,3711.0,20035.0
valoracion,256.0,4.321094,0.307591,2.90,4.2000,4.4,4.6,4.8
numero_variaciones,256.0,4.949219,7.976028,0.00,0.0000,2.0,7.0,49.0


In [123]:
outliers_valoraciones.describe(include='object').T

,count,unique,top,freq
nombre,256,246,Spike Valentino,2
descripcion,251,210,Máscara,6
nombre_marca,256,36,Benefit Cosmetics,45
nombre_categoria,256,8,Rostro,109
nombre_subcategoria,254,32,Máscara de pestañas,43


Los productos con un número de valoraciones excepcionalmente alto (outliers) tienen una mediana de 2,292.5 valoraciones, muy por encima de la mediana general de 42.5, lo que indica una altísima popularidad. No se corresponden con ser productos más baratos (mediana de 35.00, casi igual a los 34.99 generales), y presentan una valoración igual a los productos generales (mediana de 4,4), lo que sugiere una percepción positiva mantenida incluso con un volumen masivo de opiniones. En cuanto a las variaciones, su mediana es de 2.0, igual que la de los productos generales, lo que indica una disponibilidad de opciones promedio. A nivel de marcas, destacan nombres populares como Benefit Cosmetics, y en categorías, predominan productos de Rostro, especialmente la Máscara de pestañas, que aparece como la subcategoría más frecuente. En conjunto, estos productos mantienen un precio promedio en la tienda y son muy bien reconocidos, manteniendo buenas valoraciones a gran escala.

In [124]:
outliers_valoracion.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
7,Cherry Mouth,Estuche Trío de Labios,Fenty Beauty,Estuches maquillaje,Estuches de labiales,51.0,0,0.0,0
9,Lip Injection Plumping Icons,Estuche Labios,Too Faced,Estuches maquillaje,Estuches de labiales,50.0,0,0.0,0
14,Master Mascara™,Máscara voluminizadora,Makeup By Mario,Ojos,Máscara de pestañas,33.0,0,0.0,0
15,Diorshow 5 Couleurs edición limitada cannage,Paleta de sombras de ojos,Dior,Ojos,Paleta de sombras de ojos,76.0,0,0.0,4
16,Even Better Light Reflecting Primer,Prebase iluminadora,Clinique,Rostro,Primer & spray fijador,40.0,0,0.0,0


In [125]:
outliers_valoracion.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,259.0,38.490347,19.442039,7.99,27.99,36.0,46.0,144.0
numero_valoraciones,259.0,0.440154,2.122072,0.00,0.00,0.0,0.0,22.0
valoracion,259.0,0.186873,0.580615,0.00,0.00,0.0,0.0,2.6
numero_variaciones,259.0,2.637066,4.331194,0.00,0.00,0.0,4.0,26.0


In [126]:
outliers_valoracion.describe(include='object').T

,count,unique,top,freq
nombre,256,243,Moving Lights,6
descripcion,233,206,Lápiz de labios,3
nombre_marca,259,53,Sephora Collection,24
nombre_categoria,255,10,Rostro,66
nombre_subcategoria,251,43,Barra de labios,23


Los productos con valoraciones excepcionalmente bajas presentan una mediana de 0.0, muy por debajo de la mediana general de 4.4, lo que indica una percepción claramente negativa o problemas de calidad o experiencia. Además, estos productos prácticamente no tienen valoraciones (mediana también de 0), lo que refuerza la baja visibilidad o el desinterés por parte de los usuarios. A nivel de precio, su mediana (36.0) es similar a la del conjunto general (34.99), por lo que no son ni especialmente baratos ni caros. En cuanto a variaciones, la mediana es de 0, también por debajo del global (2.0), lo que sugiere menor diversidad de opciones o formatos. Predominan marcas como Sephora Collection, y las categorías más frecuentes son productos de Rostro, destacando subcategorías como Barra de labios o Lápiz de labios. En conjunto, se trata de productos con escasa interacción y muy baja valoración, lo que puede indicar desinterés, problemas de lanzamiento o experiencias de usuario deficientes.

In [127]:
outliers_variaciones.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
0,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7,50
1,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5,51
2,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1,30
13,Dior Addict,Barra de labios brillante 90 % de ingredientes...,Dior,Labios,Barra de labios,51.00,977,4.5,26
34,Vanish™ Airbrush Concealer,Corrector antiojeras,Hourglass,Rostro,Antiojeras & corrector,42.00,3247,4.3,29


In [128]:
outliers_variaciones.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,114.0,41.629386,16.368728,4.99,32.0,41.49,51.0,99.0
numero_valoraciones,114.0,1048.956140,2804.771208,0.00,20.0,69.50,585.5,17079.0
valoracion,114.0,4.103509,1.064880,0.00,4.1,4.30,4.6,5.0
numero_variaciones,114.0,25.885965,9.463258,16.00,19.0,23.00,30.0,60.0


In [129]:
outliers_variaciones.describe(include='object').T

,count,unique,top,freq
nombre,114,110,Rouge Is Not My Name,2
descripcion,112,81,Corrector,12
nombre_marca,114,34,Sephora Collection,13
nombre_categoria,114,4,Rostro,82
nombre_subcategoria,114,14,Base de maquillaje,51


Los productos con un número inusualmente alto de variaciones (mediana de 23 frente a 2 en el conjunto general) destacan por ofrecer una gran diversidad de tonos, formatos o versiones. A pesar de su complejidad de catálogo, mantienen una valoración sólida (mediana de 4.3, similar al total), lo que indica que la variedad no compromete la calidad percibida. Además, presentan un volumen notable de valoraciones (mediana de 69.5 frente a 42.5 en general), lo que sugiere buena aceptación por parte de los usuarios. Su precio (mediana de 41.49) es algo superior al promedio del catálogo, posiblemente asociado a la amplitud de oferta. Estos productos se concentran sobre todo en la categoría de Rostro, especialmente en la subcategoría de Base de maquillaje, y destacan marcas como Sephora Collection. En conjunto, se trata de productos versátiles y bien valorados, con alta interacción del cliente.

In [87]:
df.describe(include="object").T

,count,unique,top,freq
url_producto,1822,1822,https://www.sephora.es/p-640056.html,1
nombre,1819,1646,Moving Lights,14
descripcion,1724,1204,Paleta de sombras de ojos,44
nombre_marca,1822,79,Sephora Collection,198
nombre_categoria,1813,13,Rostro,627
nombre_subcategoria,1789,48,Máscara de pestañas,148


### Valores representativos de las variables categóricas

In [130]:
numero_marcas = df["nombre_marca"].nunique()
print(f"Sephora cuenta con {numero_marcas} marcas diferentes")
numero_categorias = df["nombre_categoria"].nunique()
print(f"Sephora cuenta con {numero_categorias} categorias diferentes")
numero_subcategorias = df["nombre_subcategoria"].nunique()
print(f"Sephora cuenta con {numero_subcategorias} subcategorias diferentes")

Sephora cuenta con 79 marcas diferentes
Sephora cuenta con 13 categorias diferentes
Sephora cuenta con 48 subcategorias diferentes


### Recuento de productos por marca

In [143]:
df_marca_count = df.groupby('nombre_marca').size().reset_index(name='count')
df_marca_count['%/Total'] = ((df_marca_count['count'] / numero_total_productos) * 100).round(2)
df_marca_count = df_marca_count.sort_values(by='%/Total', ascending=False)
df_marca_count.head(5)

,nombre_marca,count,%/Total
63,Sephora Collection,198,10.87
5,Benefit Cosmetics,86,4.72
10,Charlotte Tilbury,77,4.23
42,Make Up For Ever,66,3.62
15,Dior,65,3.57


Sephora Collection es la marca que más presencia tiene en número de productos ofertados en Sephora (cuenta con el 10,87% de los productos), seguida de Benefit Cosmetics y Charlotte Tilbury (4,72% y 4,23% respectivamente). Por el lado contrario, marcas como The Ordinary, Urban Decay, YEPODA y otras más, cuentan con tan solo el 0,05% de productos ofertados en Sephora.

### Recuento de productos por categoria 

In [145]:
df_cat_count = df.groupby('nombre_categoria').size().reset_index(name='count')
df_cat_count['%/Total'] = ((df_cat_count['count'] / numero_total_productos) * 100).round(2)
df_cat_count = df_cat_count.sort_values(by='%/Total', ascending=False)
df_cat_count.head(5)

,nombre_categoria,count,%/Total
10,Rostro,627,34.41
4,Ojos,372,20.42
3,Labios,343,18.83
8,Pinceles & accesorios,167,9.17
0,Cejas,100,5.49


La categoría con más presencia en Sephora es el Rostro con un 34,41% de productos ofertados en la tienda. Le siguen los Ojos con un 20,42% y los Labios con un 18,83%. Por el lado contrario, las categorías con menos presencia serían los Sacapuntas y Pestañas postizas con un 0,22%, los Rizadores de pestañas con un 0,16% y los Papeles matificantes con un 0,05%.

### Recuento de procuctos por subcategoria 

In [147]:
df_subcat_count = df.groupby('nombre_subcategoria').size().reset_index(name='count')
df_subcat_count['%/Total'] = ((df_subcat_count['count'] / numero_total_productos) * 100).round(2)
df_subcat_count = df_subcat_count.sort_values(by='%/Total', ascending=False)
df_subcat_count.head(5)

,nombre_subcategoria,count,%/Total
32,Máscara de pestañas,148,8.12
2,Barra de labios,126,6.92
8,Colorete,113,6.20
4,Base de maquillaje,107,5.87
42,Primer & spray fijador,104,5.71


Las subcategorías con más presencia dentro de Sephora son las Máscaras de pestañas (8,12%), las Barras de labios (6,92%) y los Coloretes (6,20%). Por el lado contrario, las que tienen menos presencia serían los Limpiadores de pinceles y las Paletas multi-usos con un 0,05% cada una.

Hasta ahora el éxito de los productos se ha estado observando por separado. Por un lado, se ha hecho hincapié en la valoración de los productos y por otro lado en el número de valoraciones por producto. Considero que para poder determinar un producto realmente exitoso se deben observar ambas variables en conjunto ya que, que un producto tenga muchas valoraciones en este caso puede indicar o que el producto gusta mucho (buena valoración) y por eso los consumidores se paran a comentar en el producto, o por el contrario, que el producto gusta poco (mala valoración) y por ello también se paran a comentarlo. Por otro lado, que un producto tenga una valoración muy alta no tiene por qué ser indicador de producto muy exitoso si tan solo cuenta con 2 reseñas por ejemplo, al igual que un producto con una valoración algo más baja no tiene por qué indicar que es peor si sin embargo este cuenta con 100 valoraciones. Por todo ello, para determinar los productos realmente exitosos se va a observar en conjunto el número de valoraciones y la valoración. 

Justificación de los umbrales para considerar productos "exitosos" en Sephora:

Para definir qué productos pueden considerarse exitosos en Sephora, he establecido criterios basados en las variables de valoración media y número de valoraciones, eligiendo umbrales informados por los estadísticos de la distribución general, pero también guiados por criterios prácticos de exploración.

- En cuanto a la valoración, he fijado un umbral en 4,5, que ya puede considerarse una valoración alta (la mediana está en 4,4). Aunque el percentil 75 está en 4,7, una puntuación de 4,5 sitúa al producto cerca del cuartil superior, reflejando una percepción positiva generalizada sin llegar al nivel de productos excepcionalmente valorados (outliers).

- Para el número de valoraciones, he establecido el umbral en 50 valoraciones. La mediana se encuentra en torno a 42, y aunque este umbral es ligeramente superior, permite asegurar que el producto ha sido evaluado por una base mínima de usuarios, sin restringir la muestra solo a aquellos pocos productos con cifras muy elevadas que podrían considerarse atípicos.

Además, dado que el objetivo es explorar y analizar un conjunto representativo y suficientemente amplio de productos exitosos, no conviene ser excesivamente restrictivo. Si se utilizaran directamente los percentiles 75 como umbrales, el resultado sería un grupo demasiado reducido de productos —cercano al comportamiento de outliers— que no reflejaría bien el éxito "habitual" o recurrente dentro del catálogo. Por tanto, este enfoque permite capturar productos que combinan buena valoración y reconocimiento, pero sin limitarse únicamente a casos excepcionales.

### Número y porcentaje de productos exitosos en Sephora

In [278]:
exitosos = df[(df["valoracion"] >= 4.5) & (df["numero_valoraciones"] >= 50)]

print("Número de productos exitosos:", len(exitosos))
print("Porcentaje de productos exitosos:", round((len(exitosos) / numero_total_productos) * 100, 2))

Número de productos exitosos: 393
Porcentaje de productos exitosos: 21.57


### Productos exitosos por marca

In [162]:
exitosos_por_marca = exitosos.groupby('nombre_marca').size().reset_index(name='count')
exitosos_por_marca['%/Total'] = ((exitosos_por_marca['count'] / len(exitosos)) * 100).round(2)
exitosos_por_marca = exitosos_por_marca.sort_values(by='%/Total', ascending=False)
exitosos_por_marca.head(5)

,nombre_marca,count,%/Total
3,Benefit Cosmetics,35,8.91
37,Sephora Collection,32,8.14
30,Natasha Denona,29,7.38
0,Anastasia Beverly Hills,27,6.87
19,Huda Beauty,23,5.85


Las marcas que cuentan con un mayor número de productos exitosos en su oferta son Benefit Cosmetics (con el 8,91% de los productos exitosos), Sephora Collection (con el 8,14%) y Natasha Denona (con el 7,38%)

### Productos exitosos por categorias

In [158]:
exitosos_por_cat = exitosos.groupby('nombre_categoria').size().reset_index(name='count')
exitosos_por_cat['%/Total'] = ((exitosos_por_cat['count'] / len(exitosos)) * 100).round(2)
exitosos_por_cat = exitosos_por_cat.sort_values(by='%/Total', ascending=False)
exitosos_por_cat.head(5)

,nombre_categoria,count,%/Total
7,Rostro,152,38.68
3,Labios,69,17.56
4,Ojos,63,16.03
6,Pinceles & accesorios,46,11.70
0,Cejas,26,6.62


Las categorías de productos que cuentan con un mayor número de productos exitosos son las mismas marcas que cuentan con mayor número de productos en la tienda por lo que eso también puede estar influenciando. En primer lugar los productos de Rostro (38,68% de los productos exitosos), seguidos de los productos de Labios (17,56%) y seguido de los de los Ojos (16,03%)

### Productos exitosos por subcategorias

In [271]:
exitosos_por_subcat = exitosos.groupby('nombre_subcategoria').size().reset_index(name='count')
exitosos_por_subcat['%/Total'] = ((exitosos_por_subcat['count'] / len(exitosos)) * 100).round(2)
exitosos_por_subcat = exitosos_por_subcat.sort_values(by='%/Total', ascending=False)
exitosos_por_subcat

,nombre_subcategoria,count,%/Total
8,Colorete,43,10.94
6,Brocha rostro,30,7.63
25,Paleta de sombras de ojos,29,7.38
2,Barra de labios,28,7.12
33,Sombras de ojos,23,5.85
19,Iluminador,19,4.83
24,Máscara de pestañas,18,4.58
4,Base de maquillaje,16,4.07
28,Polvos bronceadores,15,3.82
21,Lápiz & polvo para cejas,14,3.56


Las subcategorías de productos que cuentan con un mayor número de productos dentro de su catálogo difieren algo más de las subcategorías que contaban con mayor número de productos en general en Sephora. Está en primer lugar los Coloretes (10,94% de los productos exitosos), a continuación las Brochas de rostro (7,63%), las Paletas de sombras de ojos (7,38%) y las Barras de labios (7,12%)

In [239]:
mediana_precio_exitosos = exitosos["precio"].median()
mediana_precio_exitosos

np.float64(36.0)

Al ser la mediana del precio de los productos exitosos (36 euros) prácticamente el mismo que el de los productos de Sephora en general (37,20 euros), sugiere lo que ya se había comenzado a observar en los outliers, que precios más bajos no son la justificación del éxito en los productos. Aquellos productos con precios dentro del rango habitual de Sephora son los más exitosos.

In [279]:
mediana_variaciones_exitosos = exitosos["numero_variaciones"].median()
mediana_variaciones_exitosos

np.float64(2.0)

### Precio mediano por marcas

In [282]:
precio_mediano_marca = (df.groupby("nombre_marca")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_marca.tail()

,nombre_marca,precio
61,SVR,15.00
70,The Ordinary,14.99
9,Caudalie,14.00
63,Sephora Collection,13.99
46,Merci Handy,12.99


La marca con el precio mediano más alto es La Mer (102,50 euros), seguida de Sisley (65,50 euros) y Sephora Favorites (65,00). Por el lado contrario, las marcas con el precio medianos más bajo son Youth To The People (15,00), SVR (15,00), The Ordinary (14,99), Caudalie (14,00), Sephora Collection (13,99), Merci Handy (12,99).

In [244]:
precio_mediano_marca[precio_mediano_marca["nombre_marca"] == "Benefit Cosmetics"]

,nombre_marca,precio
5,Benefit Cosmetics,33.0


In [281]:
precio_mediano_marca[precio_mediano_marca["nombre_marca"] == "Erborian"]

,nombre_marca,precio
19,Erborian,30.0


De nuevo, que Benefit Cosmetics tenga un precio medio superior a Sephora Collection y aún así cuente con un mayor número de productos exitosos indica lo ya observado -> Un precio más bajo no tiene por qué ser justificante de los productos con mayor éxito.

### Precio mediano por categorías

In [251]:
precio_mediano_categoria = (df.groupby("nombre_categoria")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_categoria.tail()

,nombre_categoria,precio
7,Pestañas postizas,22.00
6,Papeles matificantes,20.00
9,Rizador pestañas,15.99
12,Uñas,15.99
11,Sacapuntas,10.00


Las categorías con precios medianos más altos son las Paletas de maquillaje (59,00 euros), el Rostro (40,99) y los Ojos (33,00), mientras que las categorías con precios medianos más bajos son los Rizadores de pestañas (15,99), las Uñas (15,99), y los Sacapuntas (10,00).

### Precio mediano por subcategorías

In [286]:
precio_mediano_subcategoria = (df.groupby("nombre_subcategoria")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_subcategoria.head()

,nombre_subcategoria,precio
36,Paleta rostro,64.99
34,Paleta de sombras de ojos,59.00
33,Paleta de contouring,55.00
4,Base de maquillaje,51.00
35,Paleta multi-usos,51.00


Las subcategorías con precios medianos más altos son las Paletas de rostro (64,99), las Paletas de sombras de ojos (59,00) y las Paletas de contouring (55,00). Por el lado contrario, las subcategorías con precios medianos más bajos son los Esmaltes de uñas (13,99), los Quitaesmaltes (13,99) y los Limpiadores de pinceles (8,99).

In [285]:
precio_mediano_subcategoria[precio_mediano_subcategoria["nombre_subcategoria"] == "Cremas BB & CC"]

,nombre_subcategoria,precio
11,Cremas BB & CC,39.0


### Marcas que dominan cada categoría de productos

In [189]:
marcas_por_categoria = df.groupby(["nombre_categoria", "nombre_marca"])["url_producto"].count().reset_index(name="numero_productos")
marcas_dominantes_por_categoria = marcas_por_categoria.loc[marcas_por_categoria.groupby("nombre_categoria")["numero_productos"].idxmax()]
marcas_dominantes_por_categoria

,nombre_categoria,nombre_marca,numero_productos
0,Cejas,Anastasia Beverly Hills,21
35,Desmaquillante,Collistar,5
77,Estuches maquillaje,Sephora Collection,9
127,Labios,Sephora Collection,24
168,Ojos,Sephora Collection,53
188,Paletas maquillaje,Natasha Denona,9
193,Papeles matificantes,Rem Beauty,1
194,Pestañas postizas,Huda Beauty,4
216,Pinceles & accesorios,Sephora Collection,48
219,Rizador pestañas,Dior Backstage,1


### Marcas que dominan cada subcategoría de productos

In [190]:
marcas_por_subcategoria = df.groupby(["nombre_subcategoria", "nombre_marca"])["url_producto"].count().reset_index(name="numero_productos")
marcas_dominantes_por_subcategoria = marcas_por_subcategoria.loc[marcas_por_subcategoria.groupby("nombre_subcategoria")["numero_productos"].idxmax()]
marcas_dominantes_por_subcategoria

,nombre_subcategoria,nombre_marca,numero_productos
0,Aceite desmaquillante,CHANEL,2
47,Antiojeras & corrector,Tarte,9
54,Barra de labios,Charlotte Tilbury,11
83,Barra de labios líquida,Benefit Cosmetics,3
107,Base de maquillaje,Fenty Beauty,7
142,Base de sombra de ojos,Too Faced,3
161,Brocha rostro,Sephora Collection,25
179,Bálsamo labial,Lanolips,12
198,Colorete,Benefit Cosmetics,20
241,Contouring,Sephora Collection,3


### Número de variaciones mediano por marca

In [255]:
variaciones_median_marca = (df.groupby("nombre_marca")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_marca.tail()

,nombre_marca,numero_variaciones
65,Shiseido,0.0
63,Sephora Collection,0.0
68,Supergoop!,0.0
73,Urban Decay,0.0
76,YEPODA,0.0


### Número de variaciones mediano por categoría

In [259]:
variaciones_median_cat = (df.groupby("nombre_categoria")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_cat.head()

,nombre_categoria,numero_variaciones
3,Labios,5.0
10,Rostro,4.0
0,Cejas,3.0
2,Estuches maquillaje,0.0
1,Desmaquillante,0.0


### Número de variaciones mediano por subcategoría

In [260]:
variaciones_median_subcat = (df.groupby("nombre_subcategoria")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_subcat.head()

,nombre_subcategoria,numero_variaciones
4,Base de maquillaje,15.0
1,Antiojeras & corrector,10.5
10,Crema de color,9.5
24,Gel uñas,9.5
30,Lápiz de labios,8.0


Ahora, con un análisis un poco más específico de Sephora Collection dentro de Sephora, va a permitir terminar de posicionar a Sephora Collection dentro de la tienda para tener esa primera visión clara de la marca y poder pasar posteriormente ya a decidir sobre el producto a lanzar en su próximo lanzamiento.

## Análisis Específico Sephora Collection

In [ ]:
query_especifica = """
select 
    p.url_producto,
    p.nombre,
    p.descripcion,
    m.nombre_marca, 
    c.nombre_categoria,
    s.nombre_subcategoria,
    h.precio,
    h.numero_valoraciones,
    h.valoracion,
    h.numero_variaciones
from productos p 
join marcas m on p.id_marca = m.id_marca
left join categorias c on p.id_categoria = c.id_categoria
left join subcategorias s on p.id_subcategoria = s.id_subcategoria
join historico h on p.id_producto = h.id_producto
where m.nombre_marca = 'Sephora Collection';
"""

In [211]:
df_especifico = ejecutar_query(query_especifica)
df_especifico.head()

,url_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
0,https://www.sephora.es/p/el-estuche-de-brochas...,El Estuche de Brochas de Iniciación,"Aplicación intuitiva, acabado perfecto",Sephora Collection,Pinceles & accesorios,Estuche de pinceles,42.99,94,4.9,0
1,https://www.sephora.es/p/get-unready*---estuch...,Get (Un)Ready*,Estuche trío belleza,Sephora Collection,Estuches maquillaje,Estuches multi-usos,24.00,0,0.0,0
2,https://www.sephora.es/p/time-to-glitter---pur...,Time To Glitter,Purpurina Instantánea 2 geles de purpurina,Sephora Collection,None,None,18.00,0,0.0,0
3,https://www.sephora.es/p/size-up---m%c3%a1scar...,Size Up,Máscara Volumen Extralargo Inmediato (Formato ...,Sephora Collection,Ojos,Máscara de pestañas,7.00,640,3.9,0
4,https://www.sephora.es/p/colorful-luminizer---...,Colorful Luminizer,Polvos iluminadores para el rostro,Sephora Collection,Rostro,Iluminador,12.99,0,0.0,3


In [213]:
df_especifico.shape

(198, 10)

Como ya se sabía del análisis general, Sephora Collection cuenta con 198 productos en su catálogo.

In [214]:
info_reporte(df_especifico)

,Numero_nulos,Porcentaje_nulos,Tipo_dato
url_producto,0,0.00,object
nombre,0,0.00,object
descripcion,21,10.61,object
nombre_marca,0,0.00,object
nombre_categoria,2,1.01,object
nombre_subcategoria,5,2.53,object
precio,0,0.00,float64
numero_valoraciones,0,0.00,int64
valoracion,0,0.00,float64
numero_variaciones,0,0.00,int64


De estos 198 productos, 21 carecen de descripción, 2 de categoría y 5 de subcategoría.

In [215]:
df_especifico["url_producto"].duplicated().sum()

np.int64(0)

De los 7 duplicados que se observaron, ninguno de ellos pertenece a la marca de Sephora Collection por lo que en este caso no hay que eliminar ningún duplicado.

In [216]:
df_especifico.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,198.0,15.846061,7.564304,3.99,11.99,13.99,16.99,59.99
numero_valoraciones,198.0,166.823232,542.807371,0.00,6.25,26.00,97.75,5428.00
valoracion,198.0,3.914141,1.406259,0.00,4.00,4.40,4.70,5.00
numero_variaciones,198.0,3.484848,7.258622,0.00,0.00,0.00,4.00,48.00


Descriptivos de las variables numéricas: 
- Precio: Los productos de Sephora Collection tienen un precio claramente más accesible dentro del catálogo de Sephora. Oscilan entre 3,99 € y 59,99 €, con una media de 15,85 € y una mediana aún más baja, de 13,99 €. Esto refleja una distribución ligeramente sesgada a la derecha, donde la mayoría de los productos tienen precios bajos, pero algunos pocos alcanzan precios más elevados, aunque muy por debajo del rango general de la tienda. El 25 % de los productos cuesta menos de 11,99 €, el 50 % menos de 13,99 €, y el 75 % menos de 16,99 €, lo que confirma la orientación de esta marca hacia productos de gama económica.
- Número de valoraciones: La cantidad de valoraciones varía significativamente, desde 0 hasta 5.428. Sin embargo, la media (166,82) está muy por encima de la mediana (26), lo que indica una distribución altamente sesgada a la derecha. Esto significa que mientras algunos productos son muy populares, con miles de valoraciones, la mayoría recibe muchas menos. Un 25 % tiene menos de 6,25 valoraciones y solo una cuarta parte supera las 97,75, lo que refleja una popularidad desigual entre productos, aunque existen casos destacados de gran visibilidad.
- Valoración: Las puntuaciones de los productos oscilan entre 0 y 5. La media es de 3,91, mientras que la mediana es de 4,40, lo que indica una ligera asimetría hacia la izquierda. Esto implica que la mayoría de los productos tienen buenas valoraciones, aunque algunos con puntuaciones muy bajas tiran la media hacia abajo. El 25 % de los productos tiene una valoración inferior a 4, el 50 % no supera los 4,4, y el 75 % se sitúa por debajo de 4,7. Esto sugiere que, en general, los productos de Sephora Collection son bien recibidos por los clientes.
- Número de variaciones: En cuanto a la variedad de opciones (como tonos o tamaños), los productos de Sephora Collection muestran una distribución muy desigual. Aunque el promedio de variaciones por producto es de 3,48, la mediana es 0, lo que indica que más de la mitad de los productos no presenta ninguna variación. El 25 % tiene 0 variaciones, el 75 % no supera las 4, y unos pocos productos alcanzan hasta 48 variaciones. Esto refleja que aunque la mayoría de los productos son simples en su oferta, existen algunos casos específicos con una gran diversidad.

### Valores representativos de las variables numéricas

In [272]:
mediana_precio_espec = df_especifico["precio"].median()
print(f"Mediana precio: {mediana_precio_espec}")
mediana_num_valoraciones_espec = df_especifico["numero_valoraciones"].median()
print(f"Mediana numero valoraciones: {mediana_num_valoraciones_espec}")
mediana_valoracion_espec = df_especifico["valoracion"].median()
print(f"Mediana valoracion: {mediana_valoracion_espec}")
mediana_num_variaciones_espec = df_especifico["numero_variaciones"].median()
print(f"Mediana numero variaciones: {mediana_num_variaciones_espec}")

Mediana precio: 13.99
Mediana numero valoraciones: 26.0
Mediana valoracion: 4.4
Mediana numero variaciones: 0.0


In [217]:
df_especifico.describe(include="object").T

,count,unique,top,freq
url_producto,198,198,https://www.sephora.es/p/el-estuche-de-brochas...,1
nombre,198,169,Moving Lights,14
descripcion,177,153,"Aplicación intuitiva, acabado perfecto",16
nombre_marca,198,1,Sephora Collection,198
nombre_categoria,196,11,Ojos,53
nombre_subcategoria,193,40,Brocha rostro,25


### Valores representativos de las variables categóricas

In [ ]:
numero_categorias_espec = df_especifico["nombre_categoria"].nunique()
print(f"Sephora Collection cuenta con {numero_categorias_espec} categorias diferentes")
numero_subcategorias_espec = df_especifico["nombre_subcategoria"].nunique()
print(f"Sephora Collection cuenta con {numero_subcategorias_espec} subcategorias diferentes")

Sephora Collection cuenta con 11 categorias diferentes
Sephora Collection cuenta con 40 subcategorias diferentes


In [226]:
categorias_totales = set(df["nombre_categoria"].dropna().unique())
categorias_sephora_collection = set(df_especifico["nombre_categoria"].dropna().unique())

categorias_no_presentes = categorias_totales - categorias_sephora_collection
print("Categorías no presentes en Sephora Collection:", categorias_no_presentes)

Categorías no presentes en Sephora Collection: {'Papeles matificantes', 'Pestañas postizas'}


In [227]:
subcategorias_totales = set(df["nombre_subcategoria"].dropna().unique())
subcategorias_sephora_collection = set(df_especifico["nombre_subcategoria"].dropna().unique())

subcategorias_no_presentes = subcategorias_totales - subcategorias_sephora_collection
print("Subcategorías no presentes en Sephora Collection:", subcategorias_no_presentes)

Subcategorías no presentes en Sephora Collection: {'Paleta multi-usos', 'Estuche para cejas', 'Estuches de ojos', 'Cremas BB & CC', 'Crema de color', 'Paleta de contouring', 'Gel uñas', 'Estuches de rostro'}


### Recuento productos por categoría

In [229]:
df_cat_count = df_especifico.groupby('nombre_categoria').size().reset_index(name='count')
df_cat_count['%/Total'] = ((df_cat_count['count'] / df_especifico.shape[0]) * 100).round(2)
df_cat_count = df_cat_count.sort_values(by='%/Total', ascending=False)
df_cat_count

,nombre_categoria,count,%/Total
4,Ojos,53,26.77
6,Pinceles & accesorios,48,24.24
8,Rostro,37,18.69
3,Labios,24,12.12
10,Uñas,12,6.06
2,Estuches maquillaje,9,4.55
0,Cejas,6,3.03
1,Desmaquillante,3,1.52
5,Paletas maquillaje,2,1.01
7,Rizador pestañas,1,0.51


Las categorías donde Sephora Collection tiene mayor presencia dentro de su catálogo de productos ofertado son en productos para Ojos (26,77% de su oferta), Pinceles & accesorios (24,24%), Rostro (18,69%) y Labios (12,12%). Por el otro extremo, en las categorías donde menor presencia tiene dentro de su oferta son en las siguientes: Desmaquillante (1,52%), Paletas maquillaje (1,01%), Rizador pestañas (0,51%) y Sacapuntas (0,51%).

### Recuento productos por subcategoría

In [268]:
df_subcat_count = df_especifico.groupby('nombre_subcategoria').size().reset_index(name='count')
df_subcat_count['%/Total'] = ((df_subcat_count['count'] / df_especifico.shape[0]) * 100).round(2)
df_subcat_count = df_subcat_count.sort_values(by='%/Total', ascending=False)
df_subcat_count

,nombre_subcategoria,count,%/Total
6,Brocha rostro,25,12.63
36,Sombras de ojos,17,8.59
26,Máscara de pestañas,15,7.58
13,Estuche de pinceles,11,5.56
17,Eyeliner,10,5.05
2,Barra de labios,9,4.55
8,Colorete,7,3.54
4,Base de maquillaje,6,3.03
30,Pincel sombra de ojos,6,3.03
34,Primer & spray fijador,6,3.03


Las subcategorías donde Sephora Collection tiene mayor presencia dentro de su catálogo de productos ofertado son en las Brochas de rostro (12,63%), las Sombras de ojos (8,59%) y las Máscaras de pestañas (7,58%). Por el otro extremo, en las subcategorías donde menor presencia tiene dentro de su oferta (vendiendo únicamente 1 producto de cada una) son en las siguientes: Base de sombra de ojos, Aceite demaquillante, Barra de labios líquida, Limpiador pinceles, Leche desmaquillante, Desmaquillante rostro, Voluminizador labios. 

### Número y porcentaje de productos exitosos en Sephora Collection

In [273]:
exitosos_espec = df_especifico[(df_especifico["valoracion"] >= 4.5) & (df_especifico["numero_valoraciones"] >= 50)]

print("Número de productos exitosos:", len(exitosos_espec))
print("Porcentaje de productos exitosos:", round((len(exitosos_espec) / df_especifico.shape[0]) * 100, 2))

Número de productos exitosos: 32
Porcentaje de productos exitosos: 16.16


Dentro de Sephora Collection los límites establecidos para determinar productos exitosos dentro de Sephora suponen límites que determinan productos algo más exitosos aún dentro de Sephora Collection ya que la valoración se corresponde, pero el número de valoraciones (50) es más alto que su mediana (26) y por tanto es un límite más exigente.

### Productos exitosos por categoría

In [274]:
exitosos_por_cat = exitosos_espec.groupby('nombre_categoria').size().reset_index(name='count')
exitosos_por_cat['%/Total'] = ((exitosos_por_cat['count'] / len(exitosos_espec)) * 100).round(2)
exitosos_por_cat = exitosos_por_cat.sort_values(by='%/Total', ascending=False)
exitosos_por_cat

,nombre_categoria,count,%/Total
3,Pinceles & accesorios,19,59.38
4,Rostro,6,18.75
2,Ojos,4,12.50
0,Desmaquillante,2,6.25
1,Labios,1,3.12


La categoría sobre la que Sephora Collection dispone de un mayor número de productos exitosos es: Pinceles & accesorios. Además, con bastante diferencia con respecto al resto, pues casi el 60% de sus productos exitosos corresponden a la misma. Por el otro lado, las categorías con un menor número de productos exitosos son: Desmaquillante (2 productos) y Labios (1 producto).

### Productos exitosos por subcategoría

In [275]:
exitosos_por_subcat = exitosos_espec.groupby('nombre_subcategoria').size().reset_index(name='count')
exitosos_por_subcat['%/Total'] = ((exitosos_por_subcat['count'] / len(exitosos_espec)) * 100).round(2)
exitosos_por_subcat = exitosos_por_subcat.sort_values(by='%/Total', ascending=False)
exitosos_por_subcat

,nombre_subcategoria,count,%/Total
1,Brocha rostro,12,37.50
5,Estuche de pinceles,4,12.50
11,Sombras de ojos,4,12.50
2,Contouring,2,6.25
9,Polvos bronceadores,2,6.25
4,Esponja de maquillaje,2,6.25
3,Desmaquillante rostro,1,3.12
0,Barra de labios,1,3.12
7,Leche desmaquillante,1,3.12
6,Iluminador,1,3.12


Las subcategorías con mayor éxito en Sephora Collection son las Brochas rostro (con el 37,50% de los productos exitosos), seguidas de los Estuches de pinceles y de las Sombras de ojos (con el 12,50% cada una). En el otro extremo, con tan solo 1 producto exitoso se encontrarían las siguientes subcategorías: Desmaquillante rostro, Barra de labios, Leche desmaquillante, Iluminador, Pincel sombra de ojos, Primer & spray fijador. 

In [276]:
mediana_precio_exitosos = exitosos_espec["precio"].median()
mediana_precio_exitosos

np.float64(18.49)

La mediana del precio de los productos exitosos de Sephora Collection está incluso por encima (18,49 euros) de la mediana del precio de los productos totales de la marca (13,99 euros), lo que sugiere que en esta marca concreta tampoco ocurre que precios más bajos en los productos sean la razón de mayor éxito en los productos. Incluso podríamos decir que precios un pelín más altos dentro de los rangos de la marca corresponden a productos muy bien valorados y que Sephora Collection se podría beneficiar de ello.

In [277]:
mediana_variaciones_exitosos = exitosos_espec["numero_variaciones"].median()
mediana_variaciones_exitosos

np.float64(0.0)

De igual forma parece que aunque Sephora Collection de por si tenga poca variedad en sus productos en cuanto a tonos o formatos, no es un problema, pues sus productos más exitosos podemos observar que mantienen el mismo perfil y por tanto un mayor número de variaciones en el producto no va a justificar un mayor éxito.

### Precio mediano por categorías

In [261]:
precio_mediano_categoria = (df_especifico.groupby("nombre_categoria")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_categoria.head()

,nombre_categoria,precio
2,Estuches maquillaje,24.00
7,Rizador pestañas,15.99
8,Rostro,15.99
5,Paletas maquillaje,15.50
6,Pinceles & accesorios,14.50


### Precio mediano por subcategorías

In [264]:
precio_mediano_subcategoria = (df_especifico.groupby("nombre_subcategoria")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_subcategoria.head()

,nombre_subcategoria,precio
27,Paleta de sombras de ojos,32.99
16,Estuches multi-usos,27.00
13,Estuche de pinceles,24.99
14,Estuches de labiales,22.00
15,Estuches de manicura,19.99


### Número de variaciones mediano por categorías

In [265]:
variaciones_median_cat = (df_especifico.groupby("nombre_categoria")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_cat.head()

,nombre_categoria,numero_variaciones
3,Labios,9.0
0,Cejas,4.5
8,Rostro,3.0
5,Paletas maquillaje,1.5
1,Desmaquillante,0.0


### Número de variaciones mediano por subcategorías

In [266]:
variaciones_median_subcat = (df_especifico.groupby("nombre_subcategoria")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_subcat.head()

,nombre_subcategoria,numero_variaciones
4,Base de maquillaje,19.0
24,Lápiz de labios,17.0
3,Barra de labios líquida,15.0
2,Barra de labios,12.0
1,Antiojeras & corrector,11.0


## Resumen Conclusiones Más Relevantes Análisis (posicionando a Sephora Collection)

### Del más General

- Cuenta con 1822 productos de maquillaje.
- Cuenta con 79 marcas, 13 categorías y 48 subcategorías.
- Valores representativos
    - Precio: 34,99
    - Número de valoraciones: 42,5
    - Valoración: 4,4
    - Número de variaciones: 2
- No se observa una relación directa entre el precio y el reconocimiento de los productos. Los productos más caros tienen buenas valoraciones, aunque un número menor de valoraciones. Los productos con mayor número de valoraciones y buenas reseñas no son necesariamente más baratos, sino que tienen un precio promedio. Por otro lado, los productos con menor éxito y visibilidad tampoco están relacionados con precios más altos, sino que también mantienen un precio promedio.
- Sephora Collection tiene la que más presencia (número de productos ofertados): 198 productos (10,87%)
- El Rostro, los Ojos y los Labios son las categorías con más presencia en Sephora. En cuanto a subcategorías son las Máscaras de pestañas, las Barras de labios y los Coloretes
- Productos exitosos en Sephora (valoración >= 4,5 y numero de valoraciones >= 50): 21,57%. Sephora Collection (segunda marca, la primera es Benefit Cosmetics) con productos más exitosos. Cuenta con 32 (8,14%)
- El Rostro, los Labios y los Ojos son las categorías con más presencia en los productos exitosos. En cuanto a subcategorías son los Coloretes, Brochas rostro, Paletas de sombras de ojos y Barras de labios.
- Sephora Collection se posiciona entre las 5 marcas más baratas (precio mediano de 13,99 euros)
- Las categorías con precios medianos más altos son las Paletas de maquillaje, el Rostro y los Ojos, mientras que las categorías con precios medianos más bajos son los Rizadores de pestañas, Uñas y Sacapuntas.
- Las subcategorías con precios medianos más altos son las Paletas de rostro, las Paletas de sombras de ojos y las Paletas de contouring. Por el lado contrario, las subcategorías con precios medianos más bajos son los Esmaltes de uñas, los Quitaesmaltes y los Limpiadores de pinceles. 
- Sephora Collection domina en número de productos ofertados en Sephora en las siguientes categorías (4/13): Estuches maquillaje, Labios, Ojos, Pinceles & accesorios. Y en las siguientes subcategorías (13/48): Brocha Rostro, Contouring, Esmalte de uñas, Estuche de pinceles, Estuches de labiales, Estuches de manicura, Estuches multi-usos, Eyeliner, Limpiador pinceles, Máscara de pestañas, Pestañas postizas, Pincel sombra de ojos, Sombras de ojos. 
- Sephora Collection se posiciona entre las marcas que como número mediano de variaciones cuenta con 0 (es decir, muy escasas opciones en la mayoría de sus productos).
- Dentro de las categorías, las únicas 3 que disponen de un número mediano de variaciones distinto de 0 son los Labios, Rostro y Cejas. Dentro de las subcategorías, las que disponen de un mayor número de variaciones mediano en sus productos son: Bases de maquillaje, Antiojeras & corrector, Crema de color, Gel de uñas.

### Del más Específico 

- Valores representativos: 
    - Precio: 13,99
    - Número de valoraciones: 26
    - Valoración: 4,4
    - Número de variaciones: 0
- Cuenta con 11 categorías y 40 subcategorías. No tiene presencia en las categorías de Papeles matificantes y Pestañas postizas. No tiene presencia en las subcategorías de Paleta multi-usos, Estuche para cejas, Estuches de ojos, Cremas BB & CC, Crema de color, Paleta de contouring, Gel uñas y Estuches de rostro.
- Las categorías donde cuenta con mayor % de productos son: Ojos, Pinceles & accesorios, Rostro y Labios. Donde cuenta con un menor % son: Desmaquillante, Paletas maquillaje, Rizador pestañas, Sacapuntas.
- Las subcategorías donde cuenta con mayor % de productos son: Brochas rostro, Sombras de ojos, Máscaras de pestañas. Donde cuenta con un menor % son: Base de sombra de ojos, Aceite demaquillante, Barra de labios líquida, Limpiador pinceles, Leche desmaquillante, Desmaquillante rostro, Voluminizador labios. 
- 32 productos exitosos. Suponen el 8,14% dentro de Sephora y el 16,16% dentro del total de Sephora Collection. La categoría que con diferencia se lleva un mayor número de productos exitosos dentro de la marca es: Pinceles & accesorios, mientras que las que menos son: Desmaquillante y Labios. En cuanto a las subcategorías, las que más productos exitosos son primero la Brocha rostro, seguida de Estuches de pinceles y Sombras de ojos. Por el lado contrario estarían: Desmaquillante rostro, Barra de labios, Leche desmaquillante, Iluminador, Pincel sombra de ojos, Primer & spray fijador. 
- Productos exitosos. Precio: 18,49 euros. Número de variaciones: 0. Un menor precio o mayores variaciones de producto no justifican mayor éxito en sus productos. 

En un dashboard esto mismo del análisis más específico para Sephora Collection podría hacerse para todas las marcas de forma muy sencilla y rápida con el uso de gráficos a partir del análisis más general y a través de emplear filtros.

justificar que no voy a poder hacer mucho hincapie en las caracteristicas de formatos etc porque no son correctas ni en la propia pagian web 

preguntar si esta bien que me este fijando mucho en las medianas (creo que representan mejor)

## Explorar detalles sobre las subcategorías de los productos a lanzar

In [311]:
df_brochas_rostro = df[df["nombre_subcategoria"] == "Brocha rostro"]
df_brochas_rostro.head()

,url_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
21,https://www.sephora.es/p/hoola-wavemaker-%e2%8...,Hoola Wavemaker – Brocha para bálsamo bronceador,None,Benefit Cosmetics,Pinceles & accesorios,Brocha rostro,38.0,0,0.0,0
109,https://www.sephora.es/p/brocha-duo-mejillas--...,Brocha Dúo Mejillas,"Aplicación Intuitiva, Acabado Perfecto",Sephora Collection,Pinceles & accesorios,Brocha rostro,20.0,60,4.8,0
143,https://www.sephora.es/p/brush---brocha-conica...,Brush,Brocha cónica para polvos,KOSAS,Pinceles & accesorios,Brocha rostro,39.0,0,0.0,0
144,https://www.sephora.es/p/brush---brocha-facial...,Brush,Brocha facial,KOSAS,Pinceles & accesorios,Brocha rostro,25.0,5,5.0,0
146,https://www.sephora.es/p/brush---brocha-bisela...,Brush,Brocha biselada para colorete,KOSAS,Pinceles & accesorios,Brocha rostro,31.0,0,0.0,0


In [317]:
df_brochas_espec = df_brochas_rostro[df_brochas_rostro["nombre_marca"] == "Sephora Collection"]
df_brochas_espec.describe().T.loc["precio"]

count    25.000000
mean     17.590400
std       6.318122
min      11.990000
25%      11.990000
50%      14.990000
75%      24.990000
max      34.990000
Name: precio, dtype: float64

In [315]:
df_brochas_espec_exito = df_brochas_rostro[
    (df_brochas_rostro["nombre_marca"] == "Sephora Collection") &
    (df_brochas_rostro["numero_valoraciones"] >= 50) &
    (df_brochas_rostro["valoracion"] >= 4.5)
]
df_brochas_espec_exito.describe().T.loc["precio"]

count    12.000000
mean     21.324167
std       6.664970
min      11.990000
25%      15.990000
50%      22.495000
75%      24.990000
max      34.990000
Name: precio, dtype: float64

In [316]:
df_brochas_espec_sinexito = df_brochas_rostro[
    (df_brochas_rostro["nombre_marca"] == "Sephora Collection") &
    ((df_brochas_rostro["numero_valoraciones"] < 50) |
    (df_brochas_rostro["valoracion"] < 4.5))
]
df_brochas_espec_sinexito.describe().T.loc["precio"]

count    13.000000
mean     14.143846
std       3.508232
min      11.990000
25%      11.990000
50%      12.990000
75%      13.990000
max      24.990000
Name: precio, dtype: float64

In [318]:
df_brochas_general = df_brochas_rostro[df_brochas_rostro["nombre_marca"] != "Sephora Collection"]
df_brochas_general.describe().T.loc["precio"]

count    79.000000
mean     44.948734
std      15.790257
min      18.000000
25%      31.000000
50%      45.000000
75%      58.500000
max      95.000000
Name: precio, dtype: float64

In [319]:
df_brochas_general_exito = df_brochas_rostro[
    (df_brochas_rostro["nombre_marca"] != "Sephora Collection") &
    (df_brochas_rostro["numero_valoraciones"] >= 50) &
    (df_brochas_rostro["valoracion"] >= 4.5)
]
df_brochas_general_exito.describe().T.loc["precio"]

count    19.000000
mean     49.421053
std      14.507913
min      24.000000
25%      39.000000
50%      46.000000
75%      62.000000
max      78.000000
Name: precio, dtype: float64

In [320]:
df_brochas_general_sinexito = df_brochas_rostro[
    (df_brochas_rostro["nombre_marca"] != "Sephora Collection") &
    ((df_brochas_rostro["numero_valoraciones"] < 50) |
    (df_brochas_rostro["valoracion"] < 4.5))
]
df_brochas_general_sinexito.describe().T.loc["precio"]

count    60.000000
mean     43.532500
std      16.029056
min      18.000000
25%      29.000000
50%      40.495000
75%      55.500000
max      95.000000
Name: precio, dtype: float64

Lo que hay que tener en cuenta entonces es que Sephora Collection es la marca más presente en número de productos dentro de Sephora (el 10,87% de los productos de Sephora corresponden a Sephora Collection), que todavía le quedan cubrir 8/48 subcategorías pero que en 13/48 es la marca que domina en número de productos. Además, que se caracteriza por sus precios competitivos (una de las marcas más baratas de Sephora) y que de forma general está bastante bien valorada. De hecho, el 8,14% (32 productos) de los productos exitosos de Sephora corresponden a Sephora Collection, cifra donde únicamente le supera la marca de Benefit Cosmetics con el 8,91%. De forma general, se observa Sephora Collection como una buena marca en Sephora que dispone de buena relación calidad-precios y que a ella acudirán aquellos clientes que quieran productos más económicos y que ocasionalmente podrán acudir a productos también muy buenos pero más caros de otras marcas que también dispone de ellas Sephora.

tengo que ver aqui como explico lo de que parece que un menor precio o mayores variaciones no es la razon principal a justificar un producto mas exitoso pero claro que esa informacion tambien se ve condicionada porque por ejemplo sephora collection como numero de variaciones le aparece siempre 0 pero porque tambien de lo que mas vende y mas exito tiene son las brochas que es imposible que vaya a tener variaciones entonces tambien dependiendo de la subcategoria de la que mas se venda y dentro de los umbrales que suela tener de variaciones y de precios 

En la decisión sobre el próximo lanzamiento de producto de Sephora se pueden dar 3 enfoques: 
- Seguro: Un lanzamiento seguro sería lanzar una nueva brocha de rostro. 
    - Las brochas de rostro son de por sí la subcategoría donde Sephora Collection vende más (el 12,63% de sus productos- 25 productos). 
    - Son la subcategoría donde dispone de un mayor número de productos exitosos (el 37,50% de sus productos exitosos- 12 productos). Es decir, el 48% de las brochas de rostro de Sephora Collection resultan exitosas.
    - Además, es también la segunda subcategoría con mayor número de productos exitosos en Sephora en general (el 7,63% de los productos exitosos- 30 productos), de donde casi la mitad (12 de ellos) pertenecen a Sephora Collection. 
    - Por tanto, ya de por sí Sephora Collection tiene mucho éxito en esta subcategoría y es ir a asegurar. Además, es una subcategoría donde las variaciones de producto son de 0 y puede centrarse en el diseño de un producto único a un precio que continúe siendo competitivo, ya que lo que se viene observando es que las brochas de rostro en Sephora Collection van desde 11 hasta 34,99 euros, y las brochas en el resto de Sephora (sin contar las de Sephora Collection) van desde 18 hasta 95 euros. Si que es verdad que las brochas que tienen éxito en Sephora Collection presentan un precio mediano superior a las que no tienen éxito (22,5 frente a 12,99 euros), y lo mismo ocurre para el resto de marcas de Sephora (46 frente a 40,5 euros), con lo que, dentro de los rangos de Sephora Collection puede optar por un precio que tire más hacía el máximo que hacía el mínimo, tratando de mantener como mínimo la calidad de sus otras brochas (que ya se conocen que se valoran muy bien), ya que se ve que con buena calidad el precio no resulta un impedimento muy grande en esta subcategoría a la hora de valorar el producto. 

- Arriesgado: 

- De mejora: 